In [1]:
file_names = ["text_file_1.txt", "text_file_2.txt", "text_file_3.txt"]

In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Read the text from the files and encode them into vectors
text_embeddings = []
for file_name in file_names:
    with open(file_name, 'r') as file:
        text = file.read()
        embedding = model.encode(text)
        text_embeddings.append(embedding)

# Convert the embeddings to a numpy array
embeddings = np.array(text_embeddings)


C:\Users\sriam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Users\sriam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transfo

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\sriam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sriam\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
import faiss

# Create the FAISS index
dimension = embeddings.shape[1]  # Dimension of the embedding
index = faiss.IndexFlatL2(dimension)  # Using L2 (Euclidean) distance
index.add(embeddings)  # Add the embeddings to the index


In [4]:
def retrieve_relevant_text(query, top_k=1):
    # Encode the query into the same vector space
    query_embedding = model.encode([query])
    
    # Use FAISS to find the closest embeddings
    distances, indices = index.search(np.array(query_embedding), top_k)
    
    # Retrieve the corresponding texts
    retrieved_texts = [open(file_names[i], 'r').read() for i in indices[0]]
    return retrieved_texts

# Test retrieval
query = "What is machine learning?"
relevant_texts = retrieve_relevant_text(query)
print("Relevant Texts:")
for text in relevant_texts:
    print(text)


Relevant Texts:
Text file 1: Machine learning is a branch of artificial intelligence. It focuses on building algorithms that can learn from data and make predictions or decisions without explicit programming.


In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model_gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")

# Function to generate a response using GPT-2 based on retrieved text
def generate_response(query, retrieved_texts):
    # Combine the query with the retrieved text
    context = " ".join(retrieved_texts) + " " + query
    
    # Tokenize the context
    inputs = tokenizer.encode(context, return_tensors="pt", max_length=1024, truncation=True)
    
    # Generate a response
    outputs = model_gpt2.generate(inputs, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Get the relevant text and generate a response
retrieved_texts = retrieve_relevant_text(query)
response = generate_response(query, retrieved_texts)
print("Chatbot Response:", response)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\sriam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sriam\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Chatbot Response: Text file 1: Machine learning is a branch of artificial intelligence. It focuses on building algorithms that can learn from data and make predictions or decisions without explicit programming. What is machine learning? Machine Learning is the process of learning from a set of data. The data is then used to build a model of the data, and then it is used in a machine-learning algorithm to predict the future. Machine-learned data can be used for many different things, including prediction of future events, prediction for future weather, or prediction about the weather.

Machine learning can also be applied to other fields, such as medicine, medicine-related research, medical research and medical-research related research. For example, machine vision is an area of machine intelligence
